# Notebook 29 - Store Dashboard Exports

This notebook generates compact, dashboard-style summaries for each store, to support internal presentations or dashboards. Each store receives a 1-page overview showing:

- Recipes selected
- Products required
- Total estimated waste value saved
- Key performance indicators (KPIs)

These summaries are suitable for export to PDF or integration into PowerPoint and reporting tools.

**Objectives**
- Load the deployment summary and per-store inventory files
- Render a dashboard-style plot per store with recipes, KPIs, and charts
- Export as individual PNG images

**Inputs**
- `deployment_exports/store_deployment_summary.csv`
- `deployment_exports/per_store/deployment_store_*.csv`

**Outputs**
- `store_dashboards/store_<id>_summary.png` (one PNG per store)


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define folders
summary_file = os.path.join("deployment_exports", "store_deployment_summary.csv")
inventory_folder = os.path.join("deployment_exports", "per_store")
output_folder = "store_dashboards"
os.makedirs(output_folder, exist_ok=True)

# Load summary
df_summary = pd.read_csv(summary_file)
print("Loaded summary:", df_summary.shape)
display(df_summary.head())


Loaded summary: (9, 5)


,store,recipes_planned,unique_ingredients,products_to_pick,estimated_value_saved
0,1024,1,0,0,0.79
1,1058,1,0,0,0.00
2,1160,1,0,0,0.00
3,3298,1,0,0,0.00
4,3345,1,0,0,0.00


In [2]:
store_files = [
    f for f in os.listdir(inventory_folder) if f.endswith(".csv")
]

store_data = {}

for file in store_files:
    store_id = int(file.split("_")[-1].split(".")[0])
    df = pd.read_csv(os.path.join(inventory_folder, file))
    df.columns = df.columns.str.strip().str.lower()
    store_data[store_id] = df

print(f"Loaded inventories for {len(store_data)} stores.")


Loaded inventories for 9 stores.


In [3]:
for store_id, df_inv in store_data.items():
    # Get store KPIs
    kpi_row = df_summary[df_summary["store"] == store_id].squeeze()
    recipes = kpi_row["recipes_planned"]
    ingredients = kpi_row["unique_ingredients"]
    products = kpi_row["products_to_pick"]
    saved = kpi_row["estimated_value_saved"]

    # Get list of recipes
    recipe_list = df_inv["recipe"].dropna().unique().tolist()

    # Setup plot
    fig, ax = plt.subplots(figsize=(8, 5))
    plt.axis("off")
    fig.suptitle(f"Store {store_id} – Deployment Summary", fontsize=14, weight="bold")

    # Text block
    text = (
        f"Recipes Planned: {recipes}\n"
        f"Unique Ingredients: {ingredients}\n"
        f"Products to Pick: {products}\n"
        f"Estimated Value Saved: €{saved:.2f}\n\n"
        f"Recipes:\n- " + "\n- ".join(recipe_list)
    )
    ax.text(0.02, 0.98, text, fontsize=11, va="top", ha="left")

    # Save plot
    output_path = os.path.join(output_folder, f"store_{store_id}_summary.png")
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()

print(f"Exported {len(store_data)} dashboard PNGs to '{output_folder}'")


Exported 9 dashboard PNGs to 'store_dashboards'
